In [2]:
import pandas as pd
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas
from geopy.geocoders import Nominatim

## Geotagging the scraped Data

In [3]:
df = pd.read_csv('all_reviews.csv')
locations_df = pd.read_csv('locations_geotagged.csv')

In [4]:
df

,beer_name,beer_rating,brewery_location,brewery_name,review_content,reviewer_name
0,St. Peter's Ruby Red Ale,3.3,"Bungay, Suffolk",St. Peter's Brewery,"Ruby color, offwhite coam. Aroma and taste: fl...",alex_leit
1,Shepherd Neame Spitfire (Bottle),2.2,"Faversham, Kent",Shepherd Neame,Disappointing show from the lads at ‘Britain’s...,shivermetimber.gray
2,Arbor Rocketman,3.8,"Bristol, Gloucestershire",Arbor Ales,"Puszka. Mętne o żółtym kolorze. Piana biała,ob...",MarcinG
3,Shepherd Neame India Pale Ale (Bottle),3.4,"Faversham, Kent",Shepherd Neame,Bottle. Amber with bronze color and white foam...,alex_leit
4,Shepherd Neame Christmas Ale,3.3,"Faversham, Kent",Shepherd Neame,Bottle. Aroma: English hops and malts with tof...,alex_leit
...,...,...,...,...,...,...
20475,Wychwood Hobgoblin IPA (Keg and Bottle),5.0,+1.8,Wychwood Brewery (Marston's),İçimi kolay ve zengin aromalı güzel bir İngili...,purikss
20476,Fuller's Black Cab Stout (Pasteurised),5.0,+1.5,Fuller's Brewery (Asahi),Stout bira severler için özellikle fıcı altern...,purikss
20477,Fuller's Organic Honey Dew (Bottle),5.0,+2,Fuller's Brewery (Asahi),Gerçekten tatlı bira severleri için özellikle ...,purikss
20478,Ballantine Burton Ale,2.2,-1.5,Pabst Brewing Company,Bottle. Deep orange to crimson pour with a den...,BVery


In [5]:
locations_df

,locations,latitude,longitude,country
0,"Bungay, Suffolk",52.456225,1.437341,United Kingdom
1,"Faversham, Kent",51.314409,0.891189,United Kingdom
2,"Bristol, Gloucestershire",51.852767,-2.255369,United Kingdom
3,"Round Rock, Texas",30.508235,-97.678893,United States of America
4,"Brooklyn, New York",40.650104,-73.949582,United States of America
...,...,...,...,...
1378,"Vielank, Mecklenburg-Vorpommern",53.235501,11.135182,Germany
1379,"Baileys Harbor, Wisconsin",45.064993,-87.124274,United States of America
1380,"Broomfield, Colorado",39.920383,-105.069146,United States of America
1381,"Homerton, Greater London",51.547079,-0.043121,United Kingdom


In [6]:
geolocator = Nominatim(user_agent="None")

In [12]:
locations_new = pd.DataFrame(df.brewery_location[~df.brewery_location.isin(locations_df.locations)].unique(),columns=['locations'])
print(len(locations_new), "new locations to search for!")

610 new locations to search for!


In [13]:
def tryconvert(x):
        try:
            return geolocator.geocode(x).latitude
        except:
            return None
    

In [14]:
def tryconvertlong(x):
        try:
            return geolocator.geocode(x).longitude
        except:
            return None

In [16]:
locations_new["latitude"] = locations_new.locations.apply(lambda x: tryconvert(x))

In [17]:
locations_new["longitude"] = locations_new.locations.apply(lambda x: tryconvertlong(x))

In [18]:
locations_new

,locations,latitude,longitude
0,"Dischingen, Baden-Württemberg",48.698674,10.357023
1,"Alexandria, Virginia",38.805110,-77.047023
2,"Ord, Nebraska",41.603343,-98.926199
3,"Ohiowa, Nebraska",40.414436,-97.453000
4,"Saint-Bruno-de-Montarville, Quebec",45.525616,-73.339180
...,...,...,...
605,"Cheddar, Somerset",51.275259,-2.776637
606,"Bradford-On-Avon, Wiltshire",51.346715,-2.250433
607,"Harlow, Essex",51.767619,0.097489
608,"Lewis Center, Ohio",40.198396,-83.010184


In [19]:
countries = []
for row in locations_new.index:
    try:
        loc = geolocator.reverse([locations_new.iloc[row,1],locations_new.iloc[row,2]], language='en')
        countries.append(loc.raw['address']['country'])
    except:
        countries.append(None)
        
    

In [29]:
locations_new['country'] = countries

In [ ]:
locations_new

In [31]:
locations_df = pd.concat([locations_df,locations_new])
locations_df.to_csv('locations_geotagged.csv',index=False)

## Deal with Non-English Descriptions